<a href="https://colab.research.google.com/github/babuus/GPT/blob/master/GPT_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import nltk
import html
import openai
import os

In [53]:
df = pd.read_excel("./Data/MRP Latest Shoe Taxonomy Sheet_Batch 3.xlsx")
df.shape

(4688, 24)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4688 entries, 0 to 4687
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   S.No                    4688 non-null   int64  
 1   gsn                     4688 non-null   int64  
 2   website                 4688 non-null   object 
 3   Prd_Title               4687 non-null   object 
 4   Prav_Style Code         3145 non-null   object 
 5   Prav_Style Code.1       4687 non-null   object 
 6   Prd_URL                 4686 non-null   object 
 7   Primary Colour          4688 non-null   object 
 8   Secondry colour         4123 non-null   object 
 9   Use                     4688 non-null   object 
 10  Category/Style/Type     4688 non-null   object 
 11  Range/Silhouette/Style  4688 non-null   object 
 12  Collection              16 non-null     object 
 13  Upper Material          4688 non-null   object 
 14  Sole Material           4688 non-null   

In [ ]:
count = 0
dt = []
for index, row in df.iterrows():
  if type(row['Prav_Style Code']) is str and "Product details" not in row['Prav_Style Code'] and type(row["Prd_Title"]) is str:
    count = count + 1
    desc = html.unescape(row['Prav_Style Code'])
    desc = desc.split("|")[0].split("Product Code")[0]
    # df.dropna(subset = ["Heel Type"], inplace=True)
    print(count, ".", row["Prd_Title"])
    print(desc)
    try:
      dt.append({"prompt":row["Prd_Title"]+"\n"+row["Primary Colour"]+"\n"+row["Upper Material"]+"\n"+
                 row["Sole Material"]+"\n"+row["Toe Shape"]+"\n"+row["Heel Type"]+"\n"+row["Fastening"]+"\n"+
                 row["Use"]+"\n"+ row["Category/Style/Type"]+"\n"+row["Occasion"]+"\n\n###\n\n", 
                 "completion":desc})
    except:
      pass
    # if count == 12:
    #   break;

In [ ]:
#Storing df as json line
df2 = pd.DataFrame(dt)
data = df2.to_json(orient="records", lines=True)
print(data)
with open('./Data/GPT_Product_Desc.jsonl', 'w') as f:
    f.write(data)

In [16]:
#Uploading jsonl to openai
openai.api_key = os.getenv("OPENAI_API_KEY") #getting OPENAI_API_Key from env variables
openai.File.create(
  file=open("./Data/GPT_Product_Desc.jsonl"),
  purpose='fine-tune'
)

<File file id=file-OlHClowEjzlQYFcesmb17ml3 at 0x216586ba540> JSON: {
  "bytes": 1612072,
  "created_at": 1641454224,
  "filename": "GPT_Product_Desc.jsonl",
  "id": "file-OlHClowEjzlQYFcesmb17ml3",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [ ]:
# Helps finetune model with uploaded file
openai.FineTune.create(training_file="file-OlHClowEjzlQYFcesmb17ml3", model="ada")

In [ ]:
# List all the fine tuned model
openai.FineTune.list_events(id="ft-iyCoXynHSHe41ccMITygi2kr")

In [ ]:
openai.FineTune.list()

### **Shoes**

In [50]:
# Input params to pass to get the description
Code = "Kari in Black Flats"
Colour = "Black"
Material = "PU Leather"
Sole_material = "Nubuck Sheet"
Toe_Shape = " "
Heel_Type = "Cushioned Flat"
Fastening = "Slip on"
use = "Lifestyle"
Category_Style_Type = "Sneakers"
Occasion = "Casual"

In [46]:
# completion request to openai
response = openai.Completion.create(
  model="ada:ft-personal-2022-01-06-08-09-37",
  # prompt="Nike Tailwind 79\nGrey\nFabric:Mesh:Suede\nRubber\nRound\nNormal\nLace\nLifestyle\nSneakers\nCasual\n\n###\n\n",
  prompt= Code+"\n"+Colour+"\n"+Material+"\n"+Sole_material+"\n"+Toe_Shape+Heel_Type+"\n"+Fastening+"\n"+use+"\n"+Category_Style_Type+"\n"+Occasion+"\n\n###\n\n",
  temperature=0.7,
  # max_tokens=64,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [47]:
response

<OpenAIObject text_completion id=cmpl-4NKkdMqMZIJ4aYNlMOaM4ECHTrfpq at 0x216580e5270> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "Converse Kari in Black Flats. The quintessential Converse style,"
    }
  ],
  "created": 1641466659,
  "id": "cmpl-4NKkdMqMZIJ4aYNlMOaM4ECHTrfpq",
  "model": "ada:ft-personal-2022-01-06-08-09-37",
  "object": "text_completion"
}

In [41]:
response.choices[0].text

'\n\n###\n\nA sneaker that was first released in ’87,'

In [37]:
# sample output 1
wrd = """A legend on the streets, the Tailwind 79 makes a welcome return this season,
bringing its chunky silhouette back to its roots and bringing to light the original’s hidden ace.
Wrapped in cool shades with a chunky rubber outsole, the sneaker shows off all of its original details, boasting the"""

# sample output 2
wrd = """FOR SPEED. The Nike Tailwind 79 boasts a lightweight upper and breathable mesh in the upper for a fast feel andYNike Air propulsive cushioning. 
The round nubuck heel and low-cut collar provide ease of entry without compromising comfort. 
Benefits Low-cut collar provides ease of entry."""

In [38]:
len(word_tokenize(wrd))

57